# How to Remove Outliers for Machine Learning

Antes de modelar, es importante limpiar los datos para garantizar que las observaciones representan correctamente el problema. Algunas veces, se tienen datos extremos, muy lejos de los valores esperados o muy distintos de los demás valores. Estos puntos se llaman *outliers* y, a menudo, los modelos de *machine learning* pueden ser mejorados al comprender e incluso remover estos valores atípicos.

## 1. What are Outliers?

> *Definimos los outliers como muestras que están excepcionalmente lejos de la mayoría de los datos.*

Pueden tener varias causas, por ejemplo:
- Error al realizar la medición.
- Corrupción de datos.
- Es una observación real (e.g. Magnus Carlsen).

No hay una manera precisa de definir e identificar *outliers*, pues dependen de las características específicas del dataset. Sin embargo, algunos métodos estadísticos pueden ayudar en esta tarea.


## 2. Test Dataset

Se utilizará una población de 10 000 números aleatorios generados por una distribución Gaussiana. 

In [1]:
# generate gaussian data
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# summarize
print('mean=%.3f stdv=%.3f' % (mean(data), std(data)))

mean=50.049 stdv=4.994


## 3. Standard Deviation Method

Si se sabe que la distribución de los valores en la muestra es Gaussiana o casi Gaussiana, se puede usar la desviación estándar de la muestra para eliminar valores atípicos. Por ejemplo, se tienen estos rangos:

- 1 desviación estándar de la media: 68% de los datos.
- 2 desviaciones estándar de la media: 95% de los datos.
- 3 desviaciones estándar de la media: 99.7% de los datos.

Con pocos datos, se podría utilizar el segundo rango; con muchos datos, se puede utilizar el valor de 4 desviacione estándar (99.9).

Por lo general, primero se estandarizan los datos. En el siguiente ejemplo, se eliminan los valores que estén alejados más de 3 desviaciones estándar de la media:

In [2]:
# generate gaussian data
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# summarize
print('mean=%.3f stdv=%.3f' % (mean(data), std(data)))

mean=50.049 stdv=4.994


Luego, se pueden identificar los valores atípicos y eliminarlos. Al unir todo esto, se obtiene el siguiente código:

In [5]:
# identify outliers with standard deviation
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# calculate summary statistics
data_mean, data_std = mean(data), std(data)
# identify outliers
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

Identified outliers: 29
Non-outlier observations: 9971


Se pueden imaginar límites en dos dimensiones que definen una elipse si se tienen dos variables. Las muestras que aparezcan fuera de la elipse serán consideradas *outliers*. En tres dimensiones, sería un elipsoide.

## 4. Interquartile Range Method

No todos los datos son lo suficientemente normales como para ser tratados como una distribución Gaussiana. Para estos casos, se puede utilizar el rango intercuartil (IQR), que identifica *outliers* que están por debajo del primer cuartil o por encima del tercer cuartil por un factor de *k*, donde *k* típicamente es 1.5.

In [7]:
# identify outliers with interquartile range
from numpy.random import seed
from numpy.random import randn
from numpy import percentile
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# calculate interquartile range
q25, q75 = percentile(data, 25), percentile(data, 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

Percentiles: 25th=46.685, 75th=53.359, IQR=6.674
Identified outliers: 81
Non-outlier observations: 9919


Esta misma técnica se puede utilizar para datos en varias dimensiones.

## 5. Automatic Outlier Detection

En *machine learning*, una manera de detectar *outliers* es mediante el *one-class classification* (OCC). Involucra entrenar un modelo en los datos "normales" y predecir si un nuevo dato es normal o un *outlier*. 

Funciona muy bien en espacios con poca dimensionalidad, pero puede ser menos confiable conforme se agregan más dimensiones (conocido como *curse of dimensionality*).

El *local outlier factor*, o LOF acortado, es una técnica que intenta aprovechar la idea de la cercanía de vecinos para detectar outliers. La librería *scikit-learn* provee una implementación de este enfoce en la clase **LocalOutlierFactor**. 

Para demostrar esto, se utilizará el conjunto de datos *Boston_Housing_Dataset.csv". 

In [8]:
# load and summarize the dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# summarize the shape of the dataset
print(X.shape, y.shape)
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(506, 13) (506,)
(339, 13) (167, 13) (339,) (167,)


El código anterior carga el dataset, reporta el número de filas y columnas y luego el número de ejemplos asignados al dataset de entrenamiento y evaluación.

En este caso, vamos a ajustar una regresión lineal, evaluar el rendimiento del modelo al entrenarlo con el conjunto de prueba y realizar predicciones, evaluándolas con el **mean absolute error** (MAE).

In [9]:
# evaluate model on the raw dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

MAE: 3.417


Seguidamente, se puede intentar eliminar *outliers* del conjunto de datos. Se espera que los valores atípicos estén sesgando las predicciones del modelo, por lo que se podrían alcanzar mejores resultados al eliminarlos. 

Esto se puede alcanzar al definir el modelo **LocalOutlierFactor** y usarlo para realizar una predicción en el dataset, marcando cada fila como normal (1) o *outlier* (-1). 

In [10]:
# evaluate model on training dataset with outliers removed
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

(339, 13) (339,)
(305, 13) (305,)
MAE: 3.356


Como se puede apreciar, se eliminaron 34 filas con *outliers*, lo que permitió pasar de un error de 3.417 a un error de 3.356 .

## ¿Qué aprendí?
Antes de este tutorial, solo conocía la regla IQR para detectar y eliminar *outliers*, por lo que aprender sobre distintos métodos y cuándo son útiles fue muy provechoso. En particular, considero que son herramientas importantes para tener en cuenta antes de trabajar con los datos y obtener una mejor preparación de ellos.

En los siguientes modelos predictivos que deba realizar, serán de utilidad para detectar y eliminar *outliers*, lo que permitirá obtener mejores resultados en las predicciones, pues estarán más ajustadas a la gran mayoría de los datos recolectados.

## Fuentes
https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/